In [ ]:
## Load packages
import rasterio
import numpy as np
import os
from rasterio.mask import mask
import geopandas as gpd
from rasterio.transform import from_bounds


In [ ]:
## Define paths
base_dir = "path-of-clipped-BA"   # Directory containing folders 2001, 2002, etc.
output_dir = "path-for-resampled-BA"  # Directory to save the resampled rasters

In [ ]:
## Loop through each year directory
# scale factor foor resampling
scale_factor=4
for year in range(2004, 2021):
    year_dir = os.path.join(base_dir, str(year))
    
    if os.path.isdir(year_dir):
        # Year-specific output directory
        year_output_dir = os.path.join(output_dir, str(year))
        os.makedirs(year_output_dir, exist_ok=True)
        
        # Get BA file (JD)
        for filename in os.listdir(year_dir):
            if filename.endswith((".tif")) and any(suffix in filename for suffix in ["JD"]):
                file_path = os.path.join(year_dir, filename)
                
                with rasterio.open(file_path) as src:
                    # Read raster data
                    data = src.read(1)
                    profile = src.profile

                    print(profile)
        
                    # Determine new dimensions
                    new_height = data.shape[0] // scale_factor
                    new_width = data.shape[1] // scale_factor
                    print(data.shape[0])
                    print(new_height)
                    
        
                    # Create an empty array for the resampled data
                    resampled_data = np.zeros((new_height, new_width), dtype=np.float32)
        
                    # Calculate percentage of BA in resampled pixels
                    for i in range(new_height):
                        for j in range(new_width):
                            # Extract 4x4 block
                            block = data[i * scale_factor:(i + 1) * scale_factor, 
                                         j * scale_factor:(j + 1) * scale_factor]
                            
                            # Total pixels in the block
                            total_pixels = scale_factor ** 2
                            
                            # Count burned pixels (values between 1 and 366)
                            burned_pixels = np.count_nonzero((block >= 1) & (block <= 366))
                            
                            # Calculate percentage of burned pixels
                            percentage_burned = (burned_pixels / total_pixels) * 100
                            
                            # Assign percentage to resampled array
                            resampled_data[i, j] = percentage_burned
        
                    # Update profile for the new resolution
                    profile.update({
                        'height': new_height,
                        'width': new_width,
                        'transform': rasterio.transform.from_bounds(
                            *src.bounds, new_width, new_height
                        ),
                        'dtype': 'float32',
                        'nodata': 0  # Treat 0 as the no-data value
                    })
        
                    # Define output file path with modified filename, saving to the year-specific folder
                    output_filename = f"{filename.replace('.tif', '')}_PT_1km_PercentBurned.tif"
                    output_path = os.path.join(year_output_dir, output_filename)
                    
                    # Save the resampled raster
                    with rasterio.open(output_path, "w", **profile) as dest:
                        dest.write(resampled_data, 1)
                
                print(f"Saved resampled file: {output_path}")
